In [1]:
from keras.applications import DenseNet201
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.layers import Dense, Dropout, Input
from tensorflow.python.keras.models import Model

In [2]:
model = DenseNet201(weights = 'imagenet', include_top = False, input_shape = (128,128, 3))

In [3]:
from keras.models import Model, Input
from keras.layers import Conv2D, Dense, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Activation, BatchNormalization
from keras.layers import Concatenate
from keras.utils import to_categorical
from keras.callbacks import Callback
from keras.optimizers import SGD
import tensorflow as tf
import imageio
from PIL import Image
import numpy as np
import keras
import random
import math
import seaborn as sns
from glob import glob
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import pandas as pd
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import keras.backend as K


input_shape = (128, 128, 3)




image_datas = glob('../Train/*/*.jpg')
forders = os.listdir('../Train')
class_name = [forder for forder in forders if not '.' in forder]
dic = {"Normal" : 0,
       "Hollenhorst_Emboli" : 1,
      "Branch_Retinal_Artery_Occlusion" : 2,
      "Cilio-Retinal_Artery_Occlusion" : 3,
      "Branch_Retinal_Vein_Occlusion" : 4,
      "Central_Retinal_Vein_Occlusion" : 5,
      "Hemi-Central_Retinal_Vein_Occlusion" : 6,
      "Background_Diabetic_Retinopathy" : 7,
      "Proliferative_Diabetic_Retinopathy":8,
      "Arteriosclerotic_Retinopathy":9,
      "Hypertensive_Retinopathy":10,
      "Coats":11,
      "Macroaneurism" :12,
      "Choroidal_Neovascularization":13,
      "diabetic_retinopathy":15,
      "glaucomatous":16,
      "cataract":17,
      "retina_disease":18,
      "Diabetic_Macular_Edema":14}

In [4]:
class_name
from collections import defaultdict
def invert_dictionary(obj):
    inv_obj = defaultdict(list)
    for key, value in obj.items():
        inv_obj[value].append(key)
    return dict(inv_obj)
new_dic = invert_dictionary(dic)
for i in range(19):
    new_dic[i] = ''.join(new_dic[i])
name = []
for i in range(19):
    name.append(new_dic[i][0])
class_name = name
X = []
Y = []
for imagename in image_datas:
    image = Image.open(imagename)
    image = image.resize((128,128))
    image = np.array(image)
    X.append(image)
    label = imagename.split('\\')[1]
    label = dic[label]
    Y.append(label)
X = np.array(X)
Y = np.array(Y)
train_images, test_images, train_labels, test_labels = train_test_split(X,Y, test_size = 0.2, shuffle=True, random_state=33)
N_TRAIN = train_images.shape[0]
N_TEST = test_images.shape[0]

In [5]:
train_images, test_images, train_labels, test_labels = train_test_split(X,Y, test_size = 0.2, shuffle=True, random_state=33)
train_labels = train_labels[..., tf.newaxis]
test_labels = test_labels [..., tf.newaxis]

In [6]:
## pixel값을 0~1 범위로 조정
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.

In [7]:
# label을 onehot-encoding
train_labels = keras.utils.to_categorical(train_labels, 19)
test_labels = keras.utils.to_categorical(test_labels, 19)

In [8]:

# tf.newaxis : 새로운 차원을 추가하여, 다른 모양의 배열간 브로드 캐스팅을 가능하게 함

train_images.shape, train_labels.shape, test_images.shape, test_labels.shape


((2903, 128, 128, 3), (2903, 19), (726, 128, 128, 3), (726, 19))

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
buffer_size = 2903).batch(50).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(50)

In [10]:
model.summary()

Model: "densenet201"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [11]:
for image_batch, label_batch in train_dataset.take(1):
    pass
image_batch

<tf.Tensor: shape=(50, 128, 128, 3), dtype=float32, numpy=
array([[[[0.        , 0.00784314, 0.00392157],
         [0.        , 0.00784314, 0.00392157],
         [0.        , 0.00784314, 0.        ],
         ...,
         [0.00392157, 0.01176471, 0.        ],
         [0.00392157, 0.01176471, 0.        ],
         [0.00392157, 0.00392157, 0.        ]],

        [[0.        , 0.00784314, 0.00392157],
         [0.        , 0.00784314, 0.        ],
         [0.        , 0.00784314, 0.00392157],
         ...,
         [0.00392157, 0.01176471, 0.        ],
         [0.        , 0.00784314, 0.00392157],
         [0.00392157, 0.00392157, 0.        ]],

        [[0.        , 0.00784314, 0.00392157],
         [0.        , 0.00784314, 0.        ],
         [0.        , 0.00784314, 0.00392157],
         ...,
         [0.        , 0.00784314, 0.        ],
         [0.        , 0.00784314, 0.00392157],
         [0.00392157, 0.00392157, 0.        ]],

        ...,

        [[0.        , 0.00784314,

In [12]:
feature_batch = model(image_batch)

In [13]:
from keras.layers import GlobalAveragePooling2D

In [14]:
# 모델 Layer 데이터화
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Layer 추가
x = model.output
# FC Layer +
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation= 'relu')(x)
x = Dense(19, activation ='softmax')(x)

model = Model(inputs = model.input, outputs = x)

In [15]:
for layer in model.layers :
    layer.trainable = True

model.summary()

# 컴파일 옵션
model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [16]:
# 텐서보드 사용
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs_DenseNet')
]

In [17]:
history = model.fit(train_dataset, epochs=10, steps_per_epoch = 58, validation_data = test_dataset, validation_steps = 14, callbacks=callbacks)

Epoch 1/10
 1/58 [..............................] - ETA: 0s - loss: 3.1856 - accuracy: 0.0000e+00WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
58/58 [==============================] - 765s 13s/step - loss: 1.2140 - accuracy: 0.5700 - val_loss: 16.0220 - val_accuracy: 0.1343
Epoch 2/10
58/58 [==============================] - 748s 13s/step - loss: 0.9467 - accuracy: 0.6243 - val_loss: 1.5968 - val_accuracy: 0.5629
Epoch 3/10
58/58 [==============================] - 716s 12s/step - loss: 0.9148 - accuracy: 0.6204 - val_loss: 1.2690 - val_accuracy: 0.5529
Epoch 4/10
58/58 [==============================] - 709s 12s/step - loss: 0.8972 - accuracy: 0.6456 - val_loss: 2.2095 - val_accuracy: 0.4557
Epoch 5/10
58/58 [==============================] - 706s 12s/st

In [18]:
socre = model.evaluate(test_dataset)
print('\nTest Score : ',socre[0])
print('Test Accuracy : ',socre[1])

15/15 [==============================] - 23s 2s/step - loss: 1.3573 - accuracy: 0.6446

Test Score :  1.3572896718978882
Test Accuracy :  0.64462810754776
